<img style="float: left;;" src='Figures/alinco.png' /></a>

# <center> <font color= #000047> Manipulación de Datos con la librería Pandas II


<img style="float: right; margin: 0px 0px 15px 15px;" src="https://numfocus.org/wp-content/uploads/2016/07/pandas-logo-300.png" width="400px" height="400px" />

> La clase pasada comenzamos a ver las funcionalidades de la librería [pandas](https://pandas.pydata.org/). Básicamente, aprendimos a cargar datos desde archivos `.csv`, aprendimos a seleccionar subcojuntos de estos datos mediante indización (obtener ciertas filas, o ciertas columnas, o ciertas filas y columnas determinadas). Finalmente, aprendimos a filtrar datos, es decir, a seleccionar registros de la base de datos que satisfagan cierta condición.

> Hoy nos enfocaremos a ver como reunir varias bases de datos que poseen información complementaria relativa a un mismo problema, para obtener una sola tabla con la información condensada.

Referencias:
- https://www.shanelynn.ie/merge-join-dataframes-python-pandas-index-1/
___

# 0. Motivación

En cualquier situación relacionada con ciencia de datos en la vida real, no pasarán más de 10 minutos sin que aparezca la necesidad de unir (`merge` o `join`) dos bases de datos complementarias para formar tu conjunto de datos para el análisis.

La unión de DataFrames es un proceso fundamental, que cualquier analista de datos en formación debe aprender. En esta clase aprenderemos cómo hacerlo, y en el camino responderemos las siguientes preguntas:

- ¿Qué son el `merge` o `join` de dos DataFrames?

- ¿Qué son `inner`, `outer`, `left` y `right` `joins`?



## Datos para la clase

En esta clase, seguiremos trabajando con los datos de la clase pasada:

En la carpeta "data" tenemos los archivos 

- "customers_data.csv": datos propios de los clientes de la empresa, 
- "sessions_data.csv": inicios de sesión de los clientes en la plataforma web de compras (similar a amazon),
- "transactions_data.csv": transacciones asociadas a cada inicio de sesión, y
- "products_data.csv": información de los productos. 

Podemos cargar cada uno de los archivos `.csv` como DataFrames de Pandas usando la función `read_csv()` de pandas, y examinar los contenidos de cada uno con el método `head()`.

In [1]:
# Importar pandas
import pandas as pd
import numpy as np

In [2]:
# Cargar customers_data
customers = pd.read_csv('Data/customers_data.csv', index_col=0)
customers

,customer_id,zip_code,join_date,date_of_birth
0,1,60091,2011-04-17 10:48:33,1994-07-18
1,2,13244,2012-04-15 23:31:04,1986-08-18
2,3,13244,2011-08-13 15:42:34,2003-11-21
3,4,60091,2011-04-08 20:08:14,2006-08-15
4,5,60091,2010-07-17 05:27:50,1984-07-28


In [3]:
customers.shape

(5, 4)

In [5]:
# Cargar sessions_data
sesiones = pd.read_csv('Data/sessions_data.csv', index_col=0)
sesiones.head(3)

,session_id,customer_id,device,session_start
0,1,2,desktop,2014-01-01 00:00:00
1,2,5,mobile,2014-01-01 00:17:20
2,3,4,mobile,2014-01-01 00:28:10


In [7]:
sesiones.shape

(35, 4)

In [8]:
sesiones.info()

<class 'pandas.core.frame.DataFrame'>
Index: 35 entries, 0 to 34
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   session_id     35 non-null     int64 
 1   customer_id    35 non-null     int64 
 2   device         35 non-null     object
 3   session_start  35 non-null     object
dtypes: int64(2), object(2)
memory usage: 1.4+ KB


In [9]:
sesiones['device'].unique()

array(['desktop', 'mobile', 'tablet'], dtype=object)

In [10]:
# Cargar transactions_data
transacciones = pd.read_csv('Data/transactions_data.csv', index_col=0)
transacciones.head(3)

,transaction_id,session_id,transaction_time,product_id,amount
0,298,1,2014-01-01 00:00:00,5,127.64
1,2,1,2014-01-01 00:01:05,2,109.48
2,308,1,2014-01-01 00:02:10,3,95.06


In [11]:
transacciones.info()

<class 'pandas.core.frame.DataFrame'>
Index: 500 entries, 0 to 499
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   transaction_id    500 non-null    int64  
 1   session_id        500 non-null    int64  
 2   transaction_time  500 non-null    object 
 3   product_id        500 non-null    int64  
 4   amount            500 non-null    float64
dtypes: float64(1), int64(3), object(1)
memory usage: 23.4+ KB


In [12]:
# Cargar products_data
productos = pd.read_csv('Data/products_data.csv', index_col=0)
productos.head(3)

,product_id,brand
0,1,B
1,2,B
2,3,B


In [14]:
productos.shape

(5, 2)

## Bonus: formato de fechas y horas

Como vimos la vez pasada, es relevante en general un correcto manejo de las fechas y horas en las bases de datos.

Lo primero que se tiene que hacer para un correcto manejo de las fechas (y horas) es identificar las columnas o variables que contienen fechas. Por ejemplo, de la tabla `customers_data`:

In [15]:
customers

,customer_id,zip_code,join_date,date_of_birth
0,1,60091,2011-04-17 10:48:33,1994-07-18
1,2,13244,2012-04-15 23:31:04,1986-08-18
2,3,13244,2011-08-13 15:42:34,2003-11-21
3,4,60091,2011-04-08 20:08:14,2006-08-15
4,5,60091,2010-07-17 05:27:50,1984-07-28


Lo que sigue, es especificarle a pandas el formato en que vienen esas fechas con la función `to_datetime()` de pandas

In [16]:
# Ayuda en la función to_datetime
help(pd.to_datetime)

Help on function to_datetime in module pandas.core.tools.datetimes:

to_datetime(arg: 'DatetimeScalarOrArrayConvertible | DictConvertible', errors: 'DateTimeErrorChoices' = 'raise', dayfirst: 'bool' = False, yearfirst: 'bool' = False, utc: 'bool' = False, format: 'str | None' = None, exact: 'bool | lib.NoDefault' = <no_default>, unit: 'str | None' = None, infer_datetime_format: 'lib.NoDefault | bool' = <no_default>, origin: 'str' = 'unix', cache: 'bool' = True) -> 'DatetimeIndex | Series | DatetimeScalar | NaTType | None'
    Convert argument to datetime.

    This function converts a scalar, array-like, :class:`Series` or
    :class:`DataFrame`/dict-like to a pandas datetime object.

    Parameters
    ----------
    arg : int, float, str, datetime, list, tuple, 1-d array, Series, DataFrame/dict-like
        The object to convert to a datetime. If a :class:`DataFrame` is provided, the
        method expects minimally the following columns: :const:`"year"`,
        :const:`"month"`, :c

In [19]:
customers['date_of_birth']

0    1994-07-18
1    1986-08-18
2    2003-11-21
3    2006-08-15
4    1984-07-28
Name: date_of_birth, dtype: object

In [18]:
# Especificar el formato de fechas en la tabla customers_data
customers['date_of_birth'] = pd.to_datetime(customers['date_of_birth'], format = "%Y-%m-%d", errors='coerce')

0   1994-07-18
1   1986-08-18
2   2003-11-21
3   2006-08-15
4   1984-07-28
Name: date_of_birth, dtype: datetime64[ns]

In [20]:
customers['join_date'] = pd.to_datetime(customers['join_date'], format = "%Y-%m-%d %H:%M:%S", errors='coerce')
customers['join_date']

0   2011-04-17 10:48:33
1   2012-04-15 23:31:04
2   2011-08-13 15:42:34
3   2011-04-08 20:08:14
4   2010-07-17 05:27:50
Name: join_date, dtype: datetime64[ns]

Hagan esto mismo para todas las tablas:

In [22]:
# Especificar el formato de fechas en la tabla sessions
sesiones.head()

,session_id,customer_id,device,session_start
0,1,2,desktop,2014-01-01 00:00:00
1,2,5,mobile,2014-01-01 00:17:20
2,3,4,mobile,2014-01-01 00:28:10
3,4,1,mobile,2014-01-01 00:44:25
4,5,4,mobile,2014-01-01 01:11:30


In [23]:
sesiones['session_start'] = pd.to_datetime(sesiones['session_start'], format = "%Y-%m-%d %H:%M:%S", errors='coerce')
sesiones['session_start']

0    2014-01-01 00:00:00
1    2014-01-01 00:17:20
2    2014-01-01 00:28:10
3    2014-01-01 00:44:25
4    2014-01-01 01:11:30
5    2014-01-01 01:23:25
6    2014-01-01 01:39:40
7    2014-01-01 01:55:55
8    2014-01-01 02:15:25
9    2014-01-01 02:31:40
10   2014-01-01 02:47:55
11   2014-01-01 03:04:10
12   2014-01-01 03:15:00
13   2014-01-01 03:28:00
14   2014-01-01 03:41:00
15   2014-01-01 03:49:40
16   2014-01-01 04:00:30
17   2014-01-01 04:14:35
18   2014-01-01 04:27:35
19   2014-01-01 04:46:00
20   2014-01-01 05:02:15
21   2014-01-01 05:21:45
22   2014-01-01 05:32:35
23   2014-01-01 05:44:30
24   2014-01-01 05:59:40
25   2014-01-01 06:17:00
26   2014-01-01 06:34:20
27   2014-01-01 06:50:35
28   2014-01-01 07:10:05
29   2014-01-01 07:27:25
30   2014-01-01 07:42:35
31   2014-01-01 08:02:05
32   2014-01-01 08:10:45
33   2014-01-01 08:24:50
34   2014-01-01 08:44:20
Name: session_start, dtype: datetime64[ns]

In [24]:
# Especificar el formato de fechas en la tabla transactions
transacciones.head()

,transaction_id,session_id,transaction_time,product_id,amount
0,298,1,2014-01-01 00:00:00,5,127.64
1,2,1,2014-01-01 00:01:05,2,109.48
2,308,1,2014-01-01 00:02:10,3,95.06
3,116,1,2014-01-01 00:03:15,4,78.92
4,371,1,2014-01-01 00:04:20,3,31.54


In [ ]:
transacciones['transaction_time']

In [25]:
transacciones['transaction_time']= pd.to_datetime(transacciones['transaction_time'], format = "%Y-%m-%d %H:%M:%S", errors='coerce')
transacciones['transaction_time']

0     2014-01-01 00:00:00
1     2014-01-01 00:01:05
2     2014-01-01 00:02:10
3     2014-01-01 00:03:15
4     2014-01-01 00:04:20
              ...        
495   2014-01-01 08:56:15
496   2014-01-01 08:57:20
497   2014-01-01 08:58:25
498   2014-01-01 08:59:30
499   2014-01-01 09:00:35
Name: transaction_time, Length: 500, dtype: datetime64[ns]

## Volviendo al problema ...

Hay columnas o variables que relacionan las tablas. Por ejemplo:

- Los clientes pueden iniciar sesión en la plataforma cuantas veces quieran. De manera que hay una relación uno a muchos entre las tablas "customers_data" y "sessions_data", mediante la variable "customer_id".

- Cuando se efectúa una transacción se supone que se está comprando un producto. Por lo tanto hay una relación uno a muchos entre las tablas "transactions_data"  y "products_data", mediante la variable "product_id".

## Problemas

Primero, quisieramos determinar cuál es el dispositivo preferido por zona.

Luego, quisiéramos determinar cuál es la marca preferida por zona.

Para resolver lo anterior necesitamos unir ("merge" o "join") nuestros datasets en uno solo para el análisis.

___
# 1. Uniendo dos DataFrames...

> “Merging” two datasets is the process of bringing two datasets together into one, and aligning the rows from each based on common attributes or columns.

Las palabras “merge” y “join” se usan indistintamente en Pandas, y en otros lenguajes como SQL y R. En Pandas, hay métodos separados “merge” y “join”, que realizan cosas similares (personalmente uso el método "merge"), y la función "merge".

Vamos a concentrarnos en el **primer problema**. En este escenario, necesitamos llevar a cabo los siguientes pasos:

- Para cada fila en el dataset `sessions_data`, debemos hacer una nueva columna que contenga el "zip_code" respectivo de cada cliente.
- Una vez hagamos esto, obtenemos la moda de los dispositivos para cada cliente.

**¿Podemos usar un ciclo?**

Claro que si. Se podría escribir un ciclo para esta tarea. Éste iría a través de cada fila en `sessions_data`, y a cada "user_id" asignar el valor de la nueva columna con la zona respectiva.

Sin embargo, usar ciclos haría nuestra tarea mucho más lenta y plagada de más código que el necesario que si se usara la función (método) `join()`.

De forma que, para estas situaciones, **nunca usar ciclos**.

Para ver cómo podemos hacer lo anterior, veamos la ayuda de la función `merge()` de pandas:

In [26]:
# Ayuda de merge()
help(pd.merge)

Help on function merge in module pandas.core.reshape.merge:

merge(left: 'DataFrame | Series', right: 'DataFrame | Series', how: 'MergeHow' = 'inner', on: 'IndexLabel | AnyArrayLike | None' = None, left_on: 'IndexLabel | AnyArrayLike | None' = None, right_on: 'IndexLabel | AnyArrayLike | None' = None, left_index: 'bool' = False, right_index: 'bool' = False, sort: 'bool' = False, suffixes: 'Suffixes' = ('_x', '_y'), copy: 'bool | None' = None, indicator: 'str | bool' = False, validate: 'str | None' = None) -> 'DataFrame'
    Merge DataFrame or named Series objects with a database-style join.

    A named Series object is treated as a DataFrame with a single named column.

    The join is done on columns or indexes. If joining columns on
    columns, the DataFrame indexes *will be ignored*. Otherwise if joining indexes
    on indexes or indexes on a column or columns, the index will be passed on.
    When performing a cross merge, no column specifications to merge on are
    allowed.

  

Ahora, veamos como podemos añadir el código postal a la tabla `sessions_data`, usando la función `merge()` de pandas:

In [28]:
# Uso de la función merge()
sesiones_con_zip = pd.merge(left=sesiones,
                              right=customers[['customer_id', 'zip_code']],
                              on="customer_id")

sesiones_con_zip

,session_id,customer_id,device,session_start,zip_code
0,1,2,desktop,2014-01-01 00:00:00,13244
1,2,5,mobile,2014-01-01 00:17:20,60091
2,3,4,mobile,2014-01-01 00:28:10,60091
3,4,1,mobile,2014-01-01 00:44:25,60091
4,5,4,mobile,2014-01-01 01:11:30,60091
5,6,1,tablet,2014-01-01 01:23:25,60091
6,7,3,tablet,2014-01-01 01:39:40,13244
7,8,4,tablet,2014-01-01 01:55:55,60091
8,9,1,desktop,2014-01-01 02:15:25,60091
9,10,2,tablet,2014-01-01 02:31:40,13244


La función `merge()` es el objetivo principal de esta clase. Básicamente, la operación de unir dos DataFrames hace lo siguiente: 

- toma el DataFrame de la izquierda (argumento left=), 
- el DataFrame de la derecha (argumento right=),
- la columna donde se va a unir (argumento on=), y
- la forma en que se va a unir (argumento how=).

La variable en común sobre la que se hace la unión está especificada en el argumento `on`.

Con este resultado, podemos filtrar por zona y luego obtener la moda.

In [36]:
customers['zip_code'].unique()

array([60091, 13244], dtype=int64)

In [33]:
# Obtener la moda por zona
sesiones_con_zip.loc[sesiones_con_zip['zip_code']==13244, 'device'].value_counts()

device
desktop    7
tablet     3
mobile     3
Name: count, dtype: int64

In [37]:
sesiones_con_zip.loc[sesiones_con_zip['zip_code']==60091, 'device'].value_counts()

device
mobile     10
desktop     7
tablet      5
Name: count, dtype: int64

### Usando GroupBy

In [38]:
sesiones_con_zip.groupby(by='zip_code')['device'].value_counts()

zip_code  device 
13244     desktop     7
          mobile      3
          tablet      3
60091     mobile     10
          desktop     7
          tablet      5
Name: count, dtype: int64

### ¿Qué son los tipos de unión inner, left, right y outer?

En nuestro ejemplo, unimos `sessions` con `customers` sobre la columna "custumers_id". En este caso, en ambas tablas existían todos los posibles valores de "customers_id" (1, 2, 3, 4, 5).

¿Qué pasa si esta situación no se cumple?

Por ejemplo, realicemos los siguientes cambios artificiales:

In [40]:
sesiones['customer_id'].unique()

array([2, 5, 4, 1, 3], dtype=int64)

In [41]:
customers['customer_id'].unique()

array([1, 2, 3, 4, 5], dtype=int64)

In [51]:
# artificial sessions_data
art_sesiones = sesiones[sesiones['customer_id']!=2]


In [52]:
# artificial customers_data
art_customers = customers[customers['customer_id']!=5]

In [53]:
art_sesiones.head()

,session_id,customer_id,device,session_start
1,2,5,mobile,2014-01-01 00:17:20
2,3,4,mobile,2014-01-01 00:28:10
3,4,1,mobile,2014-01-01 00:44:25
4,5,4,mobile,2014-01-01 01:11:30
5,6,1,tablet,2014-01-01 01:23:25


In [54]:
art_sesiones['customer_id'].unique()

array([5, 4, 1, 3], dtype=int64)

In [55]:
art_customers['customer_id'].unique()

array([1, 2, 3, 4], dtype=int64)

Por defecto, la operación `merge()` de pandas actpua con un merge tipo "inner". Un "inner merge", guarda únicamente los valores comúnes (en la columna especificada en el argumento `on=`) de ambos DataFrames.

Por ejemplo:

In [ ]:
art_sessions

In [57]:
# inner join
art_sesiones_con_zip = pd.merge(left=art_sesiones,
                              right=art_customers[['customer_id', 'zip_code']], how='inner',
                              on="customer_id")
art_sesiones_con_zip

,session_id,customer_id,device,session_start,zip_code
0,3,4,mobile,2014-01-01 00:28:10,60091
1,4,1,mobile,2014-01-01 00:44:25,60091
2,5,4,mobile,2014-01-01 01:11:30,60091
3,6,1,tablet,2014-01-01 01:23:25,60091
4,7,3,tablet,2014-01-01 01:39:40,13244
5,8,4,tablet,2014-01-01 01:55:55,60091
6,9,1,desktop,2014-01-01 02:15:25,60091
7,11,4,mobile,2014-01-01 02:47:55,60091
8,12,4,desktop,2014-01-01 03:04:10,60091
9,13,4,mobile,2014-01-01 03:15:00,60091


In [58]:
art_sesiones_con_zip['customer_id'].unique()

array([4, 1, 3], dtype=int64)

Los otros comportamientos se pueden revisar mejor a través de ejemplos:

In [60]:
# left join
art_sesiones_con_zip= pd.merge(left=art_sesiones,
                              right=art_customers[['customer_id', 'zip_code']], how='left',
                              on="customer_id")

In [61]:
art_sesiones_con_zip['customer_id'].unique()

array([5, 4, 1, 3], dtype=int64)

In [62]:
# right join
art_sesiones_con_zip= pd.merge(left=art_sesiones,
                              right=art_customers[['customer_id', 'zip_code']], how='right',
                              on="customer_id")

In [63]:
art_sesiones_con_zip['customer_id'].unique()

array([1, 2, 3, 4], dtype=int64)

In [64]:
# outer join
art_sesiones_con_zip= pd.merge(left=art_sesiones,
                              right=art_customers[['customer_id', 'zip_code']], how='outer',
                              on="customer_id")

In [65]:
art_sesiones_con_zip['customer_id'].unique()

array([1, 2, 3, 4, 5], dtype=int64)

In [66]:
art_sesiones_con_zip

,session_id,customer_id,device,session_start,zip_code
0,4.0,1,mobile,2014-01-01 00:44:25,60091.0
1,6.0,1,tablet,2014-01-01 01:23:25,60091.0
2,9.0,1,desktop,2014-01-01 02:15:25,60091.0
3,14.0,1,tablet,2014-01-01 03:28:00,60091.0
4,18.0,1,desktop,2014-01-01 04:14:35,60091.0
5,26.0,1,tablet,2014-01-01 06:17:00,60091.0
6,27.0,1,mobile,2014-01-01 06:34:20,60091.0
7,29.0,1,mobile,2014-01-01 07:10:05,60091.0
8,NaN,2,NaN,NaT,13244.0
9,7.0,3,tablet,2014-01-01 01:39:40,13244.0


Estos comportamientos son muy intuitivos a partir de sus valores.

Así mismo, sus usos son muy intuitivos (lo sabrán cuando se enfrenten a problemas reales).

### Resolvamos el problema 2

Determinar cuál es la marca preferida por zona.

In [67]:
sesiones_con_zip

,session_id,customer_id,device,session_start,zip_code
0,1,2,desktop,2014-01-01 00:00:00,13244
1,2,5,mobile,2014-01-01 00:17:20,60091
2,3,4,mobile,2014-01-01 00:28:10,60091
3,4,1,mobile,2014-01-01 00:44:25,60091
4,5,4,mobile,2014-01-01 01:11:30,60091
5,6,1,tablet,2014-01-01 01:23:25,60091
6,7,3,tablet,2014-01-01 01:39:40,13244
7,8,4,tablet,2014-01-01 01:55:55,60091
8,9,1,desktop,2014-01-01 02:15:25,60091
9,10,2,tablet,2014-01-01 02:31:40,13244


In [68]:
productos

,product_id,brand
0,1,B
1,2,B
2,3,B
3,4,B
4,5,A


In [69]:
transacciones

,transaction_id,session_id,transaction_time,product_id,amount
0,298,1,2014-01-01 00:00:00,5,127.64
1,2,1,2014-01-01 00:01:05,2,109.48
2,308,1,2014-01-01 00:02:10,3,95.06
3,116,1,2014-01-01 00:03:15,4,78.92
4,371,1,2014-01-01 00:04:20,3,31.54
...,...,...,...,...,...
495,112,35,2014-01-01 08:56:15,5,55.42
496,111,35,2014-01-01 08:57:20,3,34.87
497,276,35,2014-01-01 08:58:25,1,10.94
498,266,35,2014-01-01 08:59:30,5,19.86


In [70]:
transacciones_con_zip = transacciones.merge(right=sesiones_con_zip, on = 'session_id', how = 'inner')
transacciones_con_zip

,transaction_id,session_id,transaction_time,product_id,amount,customer_id,device,session_start,zip_code
0,298,1,2014-01-01 00:00:00,5,127.64,2,desktop,2014-01-01 00:00:00,13244
1,2,1,2014-01-01 00:01:05,2,109.48,2,desktop,2014-01-01 00:00:00,13244
2,308,1,2014-01-01 00:02:10,3,95.06,2,desktop,2014-01-01 00:00:00,13244
3,116,1,2014-01-01 00:03:15,4,78.92,2,desktop,2014-01-01 00:00:00,13244
4,371,1,2014-01-01 00:04:20,3,31.54,2,desktop,2014-01-01 00:00:00,13244
...,...,...,...,...,...,...,...,...,...
495,112,35,2014-01-01 08:56:15,5,55.42,3,mobile,2014-01-01 08:44:20,13244
496,111,35,2014-01-01 08:57:20,3,34.87,3,mobile,2014-01-01 08:44:20,13244
497,276,35,2014-01-01 08:58:25,1,10.94,3,mobile,2014-01-01 08:44:20,13244
498,266,35,2014-01-01 08:59:30,5,19.86,3,mobile,2014-01-01 08:44:20,13244


In [71]:
marca_con_zip = productos.merge(right=transacciones_con_zip, on = 'product_id', how='inner' )
marca_con_zip

,product_id,brand,transaction_id,session_id,transaction_time,amount,customer_id,device,session_start,zip_code
0,1,B,245,1,2014-01-01 00:13:00,121.75,2,desktop,2014-01-01 00:00:00,13244
1,1,B,370,2,2014-01-01 00:20:35,106.99,5,mobile,2014-01-01 00:17:20,60091
2,1,B,496,3,2014-01-01 00:28:10,137.82,4,mobile,2014-01-01 00:28:10,60091
3,1,B,459,3,2014-01-01 00:29:15,23.20,4,mobile,2014-01-01 00:28:10,60091
4,1,B,354,3,2014-01-01 00:33:35,63.84,4,mobile,2014-01-01 00:28:10,60091
...,...,...,...,...,...,...,...,...,...,...
495,5,A,338,35,2014-01-01 08:51:55,93.71,3,mobile,2014-01-01 08:44:20,13244
496,5,A,37,35,2014-01-01 08:53:00,62.79,3,mobile,2014-01-01 08:44:20,13244
497,5,A,311,35,2014-01-01 08:55:10,24.01,3,mobile,2014-01-01 08:44:20,13244
498,5,A,112,35,2014-01-01 08:56:15,55.42,3,mobile,2014-01-01 08:44:20,13244


In [72]:
marca_con_zip.groupby(by='zip_code')['brand'].value_counts()

zip_code  brand
13244     B        149
          A         37
60091     B        247
          A         67
Name: count, dtype: int64